<a href="https://colab.research.google.com/github/gitleon8301/MY-AI-Gizmo-working/blob/main/Colab-TextGen-GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# oobabooga/text-generation-webui

After running both cells, a public gradio URL will appear at the bottom in around 10 minutes. You can optionally generate an API link.

* Project page: https://github.com/oobabooga/text-generation-webui
* Gradio server status: https://status.gradio.app/

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
# ==============================
# MY-AI-Gizmo • ULTIMATE FIX
# Fixes ui.py syntax error + launches Gradio
# Version: 2.3 ULTIMATE
# ==============================

import os
import subprocess
import shutil
import re
import time
from pathlib import Path
from google.colab import drive

# ---------- CONFIG ----------
REPO_ZIP = "https://github.com/gitleon8301/MY-AI-Gizmo-working/archive/refs/heads/main.zip"
WORK_DIR = Path("/content/text-generation-webui")
DRIVE_ROOT = Path("/content/drive/MyDrive/MY-AI-Gizmo")
# ----------------------------

print("🔧 Configuring Colab environment...")
if 'MPLBACKEND' in os.environ:
    del os.environ['MPLBACKEND']
os.environ['MPLBACKEND'] = 'Agg'
print("✓ Environment ready\n")

def sh(cmd, check=False):
    return subprocess.run(cmd, shell=True, check=check, capture_output=True, text=True)

def create_symlink(source, target):
    source = Path(source)
    target = Path(target)
    if source.exists() or source.is_symlink():
        if source.is_dir() and not source.is_symlink():
            shutil.rmtree(source, ignore_errors=True)
        else:
            source.unlink(missing_ok=True)
    source.parent.mkdir(parents=True, exist_ok=True)
    source.symlink_to(target)
    print(f"✓ Linked: {source.name}")

print("="*60)
print("🚀 MY-AI-Gizmo ULTIMATE FIX")
print("="*60)

# Step 1: Mount Drive
print("\n📁 Step 1/7: Mounting Google Drive...")
drive.mount("/content/drive", force_remount=False)
print("✓ Drive mounted")

# Step 2: Create folders
print("\n💾 Step 2/7: Preparing storage...")
folders = ["models", "loras", "training", "characters", "presets", "prompts",
           "settings", "chat-history", "instruct-history", "outputs", "images",
           "logs", "cache", "extensions", "softprompts"]
for folder in folders:
    (DRIVE_ROOT / folder).mkdir(parents=True, exist_ok=True)
print(f"✓ Created {len(folders)} folders")

# Step 3: Download repo
print("\n📥 Step 3/7: Setting up repository...")
if not WORK_DIR.exists():
    sh("rm -f /content/repo.zip")
    sh(f"wget -q -O /content/repo.zip {REPO_ZIP}")
    sh("unzip -q /content/repo.zip -d /content")
    next(Path("/content").glob("MY-AI-Gizmo-working-*")).rename(WORK_DIR)
    print("✓ Downloaded")
else:
    print("✓ Already exists")
os.chdir(WORK_DIR)

# Step 4: Create symlinks
print("\n🔗 Step 4/7: Linking to Drive...")
links = [
    ("models", "models"), ("loras", "loras"), ("training", "training"),
    ("user_data/characters", "characters"), ("user_data/presets", "presets"),
    ("user_data/prompts", "prompts"), ("user_data/settings.yaml", "settings/settings.yaml"),
    ("user_data/settings.json", "settings/settings.json"), ("user_data/chat", "chat-history"),
    ("user_data/instruct", "instruct-history"), ("outputs", "outputs"),
    ("user_data/images", "images"), ("user_data/logs", "logs"),
    ("user_data/cache", "cache"), ("user_data/extensions", "extensions"),
    ("user_data/softprompts", "softprompts"),
]
for local, drive in links:
    drive_path = DRIVE_ROOT / drive
    if drive_path.suffix:
        drive_path.parent.mkdir(parents=True, exist_ok=True)
        drive_path.touch(exist_ok=True)
    else:
        drive_path.mkdir(parents=True, exist_ok=True)
    create_symlink(WORK_DIR / local, drive_path)

# Step 5: Install
print("\n⚙️  Step 5/7: Installing dependencies...")
sh("chmod +x start_linux.sh")
sh("GPU_CHOICE=A LAUNCH_AFTER_INSTALL=FALSE INSTALL_EXTENSIONS=FALSE bash start_linux.sh")
print("✓ Installation complete")

# ========================================
# Step 6: FIX THE UI.PY SYNTAX ERROR
# ========================================
print("\n🔧 Step 6/7: Fixing ui.py syntax error...")

ui_file = WORK_DIR / "modules" / "ui.py"

if ui_file.exists():
    # Read the file
    with open(ui_file, 'r', encoding='utf-8') as f:
        content = f.read()

    # Fix the indentation error around line 67-68
    # The issue is a try block without content
    fixed_content = content

    # Pattern 1: Empty try block
    if re.search(r'try:\s*\n\s*theme = theme\.set\(', content):
        # Add pass statement after try:
        fixed_content = re.sub(
            r'(try:)\s*\n(\s*)(theme = theme\.set\()',
            r'\1\n\2    pass  # Fixed by launcher\n\2\3',
            content
        )

    # Pattern 2: Look for the actual problematic section
    lines = content.split('\n')
    for i, line in enumerate(lines):
        if i >= 66 and i <= 70:  # Around line 67
            if 'try:' in line and i+1 < len(lines):
                next_line = lines[i+1]
                # Check if next line is not indented enough
                if next_line.strip() and not next_line.startswith('    '):
                    # Fix indentation
                    indent = len(line) - len(line.lstrip())
                    lines[i+1] = ' ' * (indent + 4) + next_line.lstrip()
                    fixed_content = '\n'.join(lines)
                    break

    # Write the fixed content back
    with open(ui_file, 'w', encoding='utf-8') as f:
        f.write(fixed_content)

    print("✓ ui.py syntax fixed")
else:
    print("⚠️  ui.py not found, skipping fix")

# ========================================
# Step 7: Launch Gradio UI
# ========================================
print("\n🌐 Step 7/7: Starting Gradio UI...")
print("="*60)
print("⏳ Launching server - URLs will appear below...")
print("="*60 + "\n")

# Kill old servers
sh("pkill -f 'python server.py'", check=False)
sh("pkill -f 'gradio'", check=False)
time.sleep(2)

# Clean environment
env = os.environ.copy()
env['MPLBACKEND'] = 'Agg'
env['GRADIO_ANALYTICS_ENABLED'] = 'False'
if 'PYTHONPATH' in env:
    del env['PYTHONPATH']

# Activate the conda environment and launch
# Use the installed Python from installer_files
python_path = WORK_DIR / "installer_files" / "env" / "bin" / "python"
if not python_path.exists():
    python_path = "python"  # Fallback to system python

cmd = f"{python_path} server.py --share --listen"

print(f"🚀 Launching with: {cmd}\n")

proc = subprocess.Popen(
    cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
    text=True, bufsize=1, env=env, cwd=str(WORK_DIR)
)

local_url = None
public_url = None
shown = False
error_lines = []

for line in proc.stdout:
    print(line, end="")
    error_lines.append(line)

    # Capture URLs
    if not public_url:
        m = re.search(r"(https://[a-z0-9]+\.gradio\.live)", line)
        if m:
            public_url = m.group(1)

    if not local_url:
        m = re.search(r"Running on local URL:\s+(http://[^\s]+)", line)
        if m:
            local_url = m.group(1)
        elif "Running on" in line:
            m = re.search(r"(http://(?:127\.0\.0\.1|0\.0\.0\.0|localhost):\d+)", line)
            if m and ":5000" not in m.group(1):
                local_url = m.group(1)

    # Show success
    if (local_url or public_url) and not shown:
        print("\n" + "="*70)
        print("✨ SUCCESS! GRADIO UI IS RUNNING! ✨")
        print("="*70)
        if local_url:
            print(f"\n📍 LOCAL  : {local_url}")
            print("   ↳ Click to open in Colab")
        if public_url:
            print(f"\n🌍 PUBLIC : {public_url}")
            print("   ↳ Share this or use on phone!")
        print(f"\n💾 STORAGE: {DRIVE_ROOT}")
        print("\n" + "="*70)
        print("🎯 NEXT STEPS:")
        print("1. Click the PUBLIC URL above")
        print("2. Download a model")
        print("3. Load it in 'Model' tab")
        print("4. Start chatting in 'Chat' tab!")
        print("="*70 + "\n")
        shown = True
        error_lines = []  # Clear error tracking after success

    # Check for critical errors
    if "Error" in line or "Exception" in line or "Traceback" in line:
        if len(error_lines) > 50:
            error_lines = error_lines[-50:]  # Keep last 50 lines

# If process ended without showing URLs
if not shown and proc.poll() is not None:
    print("\n" + "="*60)
    print("⚠️  SERVER STOPPED WITHOUT LAUNCHING UI")
    print("="*60)
    print("\n📋 Last 20 lines of output:")
    for line in error_lines[-20:]:
        print(line, end="")
    print("\n" + "="*60)
    print("🔧 Troubleshooting:")
    print("1. Check the error above")
    print("2. Try restarting the Colab runtime")
    print("3. Re-run this cell")
    print("="*60)

print("\n✓ Script finished")
print(f"💾 Your data is safe in: {DRIVE_ROOT}")